## Import

In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib inline
import matplotlib as mpl
mpl.rcParams['figure.figsize'] = [14, 14]
mpl.rcParams['figure.dpi'] = 72

import matplotlib.pyplot as plt
import cv2
import json
import numpy as np
from shadow_utils import cut_mask, mask_to_shadow, create_gradient_shadow_alpha_mask, match_legs_ARAP, restore_mask

In [ ]:
img_name="timecity_01886_none"
org_img = cv2.imread(f'./svm_for_shadow_generation/images/{img_name}.jpg')
org_mask = cv2.imread(f'./svm_for_shadow_generation/masks/{img_name}.jpg')
data = json.load(open(f'./svm_for_shadow_generation/leg_pos/{img_name}.json', "r"))
leg_coordinates = np.zeros((1,4,2)) # leg positions
for i in range(4):
    leg_coordinates[0][i] = data['shapes'][i]['points'][0]

is_vertical_flip = True

In [ ]:
# Cut the mask into a squared patch (around the object) for some transformation
mask, cut_info = cut_mask(org_mask)
new_l, new_t = cut_info
new_leg_coordinates = np.copy(leg_coordinates)
new_leg_coordinates[0,:,0]-=new_l
new_leg_coordinates[0,:,1]-=new_t

# Either [-30,-20] or [-50,-40] seems ok
shadow_mask, all_matrix = mask_to_shadow(mask, new_leg_coordinates, rot_rd_low=-30,
                                         rot_rd_high=-20,
                                         is_vertical_flip=is_vertical_flip)
plt.imshow(shadow_mask)

## Using TPS code to match legs (Legacy)

In [ ]:
# from shadow_utils import match_legs_TPS
# while True:
#     shadow_mask_out = match_legs_TPS(shadow_mask, leg_coordinates, rotate_matrix, -0.05, 0.05)
#     l,r,t,b = get_mask_bbox(shadow_mask_out)
#     if l>0 and t>0 and r<shadow_mask.shape[1]-1 and b<shadow_mask.shape[0]-1:
#         shadow_mask = shadow_mask_out
#         break
# pad_warped_shadow_mask = restore_mask(org_mask, shadow_mask, cut_info)

## Using ARAP (As Rigid As Possible) to match legs

In [ ]:
image_rgb = cv2.cvtColor(255-shadow_mask, cv2.COLOR_BGR2RGB)
mask_gray = cv2.cvtColor(shadow_mask, cv2.COLOR_BGR2GRAY)
src_pts = cv2.perspectiveTransform(new_leg_coordinates, all_matrix)[0]
deformed_image = match_legs_ARAP(image_rgb, mask_gray, src_pts, new_leg_coordinates)

# Recover original image size
pad_warped_shadow_mask = restore_mask(org_mask, 255-deformed_image, cut_info)

In [ ]:
alpha = create_gradient_shadow_alpha_mask(pad_warped_shadow_mask, 
                                          rd_low=0, rd_high=10,
                                          is_vertical_flip=is_vertical_flip,
                                          speed=12, overflow=1.2, offset=120)
revert_shadow_mask=cv2.subtract(org_img,pad_warped_shadow_mask)

# Convert uint8 to float
revert_shadow_mask_f = revert_shadow_mask.astype(float)
img_f = org_img.astype(float)

# Multiply the foreground with the alpha matte
foreground_shadow = cv2.multiply(alpha, revert_shadow_mask_f)

# Multiply the background with ( 1 - alpha )
background_img = cv2.multiply(1.0 - alpha, img_f)

# Add the masked foreground and background.
shadow_on_top = cv2.add(foreground_shadow, background_img)
shadow_on_top = shadow_on_top.astype('uint8')

org_background=cv2.subtract(org_img,org_mask)
cut_obj=cv2.subtract(org_img,org_background)

final_background=cv2.subtract(shadow_on_top,org_mask)
final_img = cut_obj+final_background
# plt.imshow(shadow_on_top[...,::-1])
# plt.axis('off')
plt.imshow(final_img[...,::-1])

# idx = "flip"
# cv2.imwrite(f'{img_name}_{idx}.png', final_img)